# Preparación de datos con Pandas

Autores: Ismael Sagredo Olivenza y Fernando Carlos López Hernández

## Pandas

Pandas es el acrónimo de (Python Data Analysis Library) es un código fuente open source con licencia BSD que proporciona estructuras de datos, herramientas de análisis en python, fáciles de usar y de alto rendimiento.

**Dataframe**

Es la estuctura fundamental de Pandas. Es una especie de tabla que permite cargar información proveniente de un fichero y poder manejarla a nuestro antojo. Soporta importación directa desde csv lo que es muy útil ya que es el formato más extendido de los diferentes datasets que están publicados en internet.
El dataframe puede tener en cada columna un tipo de datos diferentes.

Vamos a trabajar con Python para tener cierta soltura con el lenguaje antes realizar los laboratorios.
Para ello vamos a realizar el siguiente ejercicios que consisten en trabajar con el datase de películas de movielens dataset que se puede descargar aquí:
https://grouplens.org/datasets/movielens/

Lo primero que haremos será cargar el fichero movies_metadata.csv usando Pandas en un dataframe.

In [ ]:
import pandas as pd
import numpy as np
movies = pd.read_csv('movies_metadata.csv')


El warning se produce porque hay algunos campos del dataset que son multicampo y no sabe de que tipo son. Vamos a establecer los tipos para que desaparezca el warning. Como nos indica que el campo que no conoce es el 10, establecemos el 10 como str.

In [ ]:

movies = pd.read_csv('movies_metadata.csv',dtype={ 10 : 'str'})

Podemos visualizar el dataframe entero usando display. Pero en muchas ocasiones nos interesa mostrar solo una parte del dataframe para que podamos ver que pinta tiene el dataframe.


In [ ]:
display(movies.head(n=4));
display(movies.tail(n=4));

Echamos un vistazo al dataframe, los principales campos que nos interesan son el id, el original_title, los géneros, pero también hay otros interesantes como el rating medio vote_ average.

Ahora cargamos el fichero de ratings ratings_small.csv

In [ ]:
ratings = pd.read_csv('ratings_small.csv')

In [ ]:
display(ratings.head(n=4));
display(ratings.tail(n=4));

En el fichero rating tenemos las valoraciones de los usuarios identificados con un id anónimo (userId) y el identificador de la palícula (movieId). De esta forma podemos relacionar ambos dataframes. Por ejemplo, si queremos buscar todas las votaciones de una película concreta podemos extraer el id de la película que queremos buscar y con ese id consultar los ratings. Buscamos por ejemplo la película "The Lation King"

In [ ]:
lion = movies[movies["title"] == "The Lion King"]
display(lion)

Pero buscar en un campo de texto de esta forma es complicado, ya que podemos estar buscando un texto y este no estar escrito exactamente igual que el almacenado (puedes poner una letra minúscula en alguna de las mayúsculas para comprobarlo). Una solución es buscar parcialmente. Pandas no lo permite hacer directamente, pero tenemos algunos trucos para conseguirlo, por ejemplo usando máscaras.

In [ ]:
tested = 'lion king'
#Hacemos una subselección de los dos campos que nos interesan y los copiamos a otro dataframe
titles=movies[["original_title","id"]].copy();
# Nos creamos una máscara con el método applymap que nos indica que campos contienen una condición que establecemos mediante una lambda
mask = titles.applymap(lambda x:  tested.lower() in str(x).lower())
# Aplicamos la mascara para encontrar que campos nos interesan usando la función any
df1 = titles[mask.any(axis=1)]
#Any nos devuelve la fila o columna (dependiendo del axis) que al menos tenga un campo a true de la máscara. Axis 1 indica
# la columna en este caso. Si queremos devolver la fila, sería axis = 0
display(df1)

Nos encuentra tres de las películas de de la saga de The Lion King. Ahora podemos identificar correctamente el criterio de búisqueda para seleccionar correctamente el que deseamos

In [ ]:
lion = movies[movies["title"] == "The Lion King"]
display(lion)

Identificada la entrada de la película que queremos buscar, ahora procedemos, usando el ID a buscar los ratings de dicha película. Para acceder al valor del id tenemos que usar la función loc que define una selección a traves de un indice, en nuestro caso el 0 que nos devolverá la fila del índice.

In [ ]:
lionId = int(lion["id"].loc[lion.index[0]])
#lionId = movies[movies["title"] == "The Lion King"]["id"]
print(lionId)
ratingLion = ratings[ratings["movieId"] == lionId]
display(ratingLion)

Ahora queremos calcular la media de las valoraciones de la película en el fichero de ratings. Para ello vamos a ejecutar la función mean()

In [ ]:
print(ratingLion["rating"].mean())

Entre los campos vemos que hay un timestamp, vamos a convertirlo a date y le cambiamos el nombre a la columna

In [ ]:
import datetime

ratings["timestamp"] = ratings["timestamp"].apply(lambda x:  datetime.datetime.fromtimestamp(x).isoformat())
ratings.rename(columns = {'timestamp':'datetime'}, inplace = True)
display(ratings)


vamos a extraer el género de la película, lo primero que vemos es que los nombres estan con comillas simples, esto provoca un error en el parseo de json, asi que debemos primero limpiar la info usando replace()

In [ ]:
genre = lion["genres"].loc[lion.index[0]];
genre = genre.replace("'", "\"")
import json
genreArray = json.loads(genre)
for g in genreArray:
    print(g["name"])

Ahora vamos a aplicar una función que convierta los géneros en una cadena separada por comas para poder usarla en nuestras consultas más cómodamente. Para ello vamos a usar la función apply que nos permite aplicar una función a una fila o columna

In [ ]:
def transform_genre(x):
    x = x.replace("'", "\"")
    xArr = json.loads(x)
    strOut = ""
    for g in xArr:
        strOut = strOut + g["name"] + ","
    strOut = strOut[0:-1]
    return strOut


movies["genres"] = movies["genres"].apply(transform_genre)


In [ ]:
display(movies)

Ahora podemos buscar todas las películas de animación y por ejemplo calcular su puntuación media usando una máscara.

In [ ]:
tested = "Animation"
maskGenre = movies.applymap(lambda x:  tested.lower() in str(x).lower())
animation = movies[maskGenre.any(axis=1)]
display(animation.head())
print(animation["vote_average"].mean())

Ahora vamos a contar aquellas películas que tenían video y ver si son más que las que no lo tienen, para ello usamos groupby. Esta función agrupa por uno o varios campos, haciendo una especie de lista de colisiones para el resto de campos, de forma que podemos luego hacer preguntas sobre la lista.

In [ ]:
numVideo = movies.groupby("video").count()
display(numVideo)

Como vemos, la función count se aplica a todos los campos de la lista de colisión. Pero podemos aplicarlo a un campo concreto, al que nos interese, para que sea más óptimo. Por ejemplo vamos a calcular el runtime medio de las películas de video y las que no.

In [ ]:
revenueVideo = movies.groupby("video")["runtime"].mean()
print(revenueVideo[revenueVideo.index[0]])
print(revenueVideo[revenueVideo.index[1]])

Cargamos el fichero credits

In [ ]:
credits = pd.read_csv('credits.csv')
display(credits.head())

Y hacemos un merge entre movies y credits por el campo id. Si el campo id es int64 convertirlo a string

In [ ]:
movies["id"] = movies["id"].astype(str)
credits["id"] = credits["id"].astype(str)
movie_credits = pd.merge(movies,credits,how='inner',on=('id'))
display(movie_credits)

Ahora podemos saber los actores de una película, pero tambien que películas ha hecho un actor.
## Ejercicio 01
Buscar todas las películas interpretadas por **Bruce Willis**. Pero nos vamos a encontrar nuevamente con problemas en el json. Hay dos estrategias posibles, limpiar el json o ir al grano. Limpiar el json será costoso y dará bastantes problemas os proponemos la segunda opción:
Hay que crear una función que convierta Json a array por espacios. La función anterior no nos vale porque este json tiene errores que hay que subsanar más allá de convertir comillas simples por dobles (por ejemplo algunos campos tiene comillas simples en el interior del campo), por lo que es mejor ir directamente a procesar los campos name: <nombre del actor> extraerlos y contruir una lista sin usar el parser de json.
    
NOTA: Esto es muy típico en los datasets, no todos los datos vienen limpios y gran parte del trabajo en IA en general y Machine Learning en particular es preprocesar los datos paraque nos sean útiles.


In [ ]:
# Escribe aqui el ejercicio 1

Vamos a filtrar todas las películas de animación que tienen homepage y vamos a crearnos un dataframe manualmente con tres columnas, el id de la película, el título y el homepage y generaremos un número aleatorio . Después, escribiremos el dataframe como csv.

In [ ]:
AnimationHomepage = animation[animation["homepage"].notnull()]
display(AnimationHomepage)
NewAnimationHomepage = pd.DataFrame({'id':AnimationHomepage['id'],
                          'homepage':AnimationHomepage["homepage"],
                          'title':AnimationHomepage["title"]})



Escribimos el dataframe

In [ ]:
NewAnimationHomepage.to_csv("NewAnimationHomepage.csv",index=False)

## Uso de sklearn para calcular una regresión lineal

Sklearn es la libreria principal que vamos a utilizar para aplicar técnicas
de aprendizaje máquina. Pero despone de multitud de herramientas que son útiles
Por ejemplo dispone de regresión lineal.

Vamos a optener la ecuación lineal que minimiza el error cuadrático medio.
Esta ecuación constará de una variable dependiente y un conjunto de variables
independientes.

Para cada x se optendrá un error concreto en función de la distancia existente
entre el punto marcado por la recta y el valor real.

In [ ]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
#cargando el dataset de diabetes que incorpora sklearn.
diabetes = datasets.load_diabetes()
#usamos solo una caracteristica
diabetes_x = diabetes.data[:,np.newaxis,2]
#dividimos el dato ente entrenamiento y validación o test.
#esto es algo recurrente cuando hacemos aprendizaje máquina.

diabetes_x_train = diabetes_x[:-20] # Todos menos los 20 ultimos.

diabetes_x_test = diabetes_x[-20:] # desde el puesto 10 empezanod por le final hasta el final

# luego cogemos las clases para obtener los valores esperados.
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

#creamos la regresión lineal:
linearreg = linear_model.LinearRegression()
linearreg.fit(diabetes_x_train, diabetes_y_train)

#Comprobamos la capacidad de predicción
diabetes_y_pred = linearreg.predict(diabetes_x_test)
print("Mostramos los valores obtenidos por la regresión lineal")
print('Coeficientes: \n', linearreg.coef_)
print("MSE: %.2f"
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
print('R2: %.2f' % r2_score(diabetes_y_test, diabetes_y_pred)) #coeficiente de regresión.


Ahora dibujamos una gráfica usando matplotlib.pyplot que muestre la matriz de puntos y al recta de regresión.

In [ ]:

import matplotlib.pyplot as plt
plt.scatter(diabetes_x_test, diabetes_y_test,  color='black')
plt.plot(diabetes_x_test, diabetes_y_pred, color='blue', linewidth=3) #en linewidth le indicamos la linea en la gráfica.
plt.xticks(())
plt.yticks(())

plt.show()

# Ejercicio 02
Realiza otra recta de regresión del dataset "Boston" (Boston house prices) que tambien podeis encontrar en sklearn, pintando su gráfica.

In [ ]:
# Escribe aqui el ejercicio 2

# Ejercicio 03
Basándote en datos aleatorios (los que quieras) crea un gráfico de tarta (pie chart) con Mapplotlib. La gráfica debe tener un título, etiquetas de cada clase, visible los porcentajes de cada clase y opcionalmente una de las clases resaltadas.

In [ ]:
# Escribe aqui el ejercicio 3